# Advanced Features

This notebook explores advanced features of the ingestor library.

## Topics Covered

- Advanced chunking strategies
- Custom metadata extraction
- Table and figure processing
- Formula extraction
- Multi-language processing
- Index management
- Batch processing patterns

In [ ]:
from ingestor import Pipeline, PipelineConfig, IndexDeploymentManager
from ingestor.config import ChunkingMode, TableRenderMode, InputMode
from dotenv import load_dotenv
import asyncio

load_dotenv()
print("✅ Imports complete")

## 1. Advanced Chunking Strategies

### Layout-Aware Chunking

Respects document structure (sections, paragraphs, tables):

In [ ]:
config = PipelineConfig.from_env()

# Layout-aware chunking
config.chunking.mode = ChunkingMode.LAYOUT
config.chunking.target_chunk_size = 1000
config.chunking.chunk_overlap = 150

# Preserve section boundaries
config.chunking.preserve_section_boundaries = True

print("Layout-aware chunking configuration:")
print(f"  Mode: {config.chunking.mode}")
print(f"  Target size: {config.chunking.target_chunk_size}")
print(f"  Preserve sections: {config.chunking.preserve_section_boundaries}")

print("\n💡 Layout-aware chunking:")
print("  - Keeps paragraphs intact")
print("  - Preserves table structures")
print("  - Respects section headers")
print("  - Better semantic coherence")

## 2. Table Processing

### Table Rendering Modes

In [ ]:
# Example: Compare table rendering modes
modes = [
    (TableRenderMode.MARKDOWN_DETAILED, "Best for search and LLMs"),
    (TableRenderMode.MARKDOWN_COMPACT, "Space-efficient"),
    (TableRenderMode.HTML, "For HTML rendering systems"),
    (TableRenderMode.TEXT, "Most compact, loses structure")
]

print("Table Rendering Modes:\n")
for mode, description in modes:
    print(f"{mode.value}:")
    print(f"  {description}\n")

# Use detailed markdown for best results
config.chunking.table_render_mode = TableRenderMode.MARKDOWN_DETAILED
print(f"✅ Using: {config.chunking.table_render_mode}")

### Table-Specific Chunking

In [ ]:
# Keep tables together in chunks
config.chunking.keep_tables_intact = True

print("Table chunking strategy:")
print(f"  Keep tables intact: {config.chunking.keep_tables_intact}")
print("\n💡 This ensures tables aren't split across chunks")
print("   Large tables may create chunks larger than target size")

## 3. Formula and Math Processing

In [ ]:
# Enable formula extraction
config.document_intelligence.features = [
    "OCR_HIGH_RESOLUTION",
    "FORMULAS",  # Extract mathematical formulas
    "LANGUAGES"
]

print("Formula extraction enabled")
print("\n💡 Formulas are extracted as LaTeX and included in chunks")
print("   Example: $E = mc^2$ or $$\\frac{a}{b}$$")

## 4. Multi-Language Processing

In [ ]:
# Process multi-language documents
config.document_intelligence.features = [
    "LANGUAGES"  # Auto-detect languages
]

# Metadata will include detected language
# Index field 'language' will be populated automatically

print("Multi-language support:")
print("  - Auto-detects document language")
print("  - Stores in 'language' field")
print("  - Supports 100+ languages")
print("\nSupported languages include:")
print("  English, Spanish, French, German, Chinese, Japanese, etc.")

## 5. Custom Metadata Extraction

Add custom metadata to documents during processing:

In [ ]:
# Example: Add custom metadata based on filename patterns
from ingestor.models import DocumentMetadata

def extract_custom_metadata(filename: str) -> dict:
    """Extract custom metadata from filename."""
    metadata = {}
    
    # Example: Extract product from filename
    # "pacemaker-5000-manual.pdf" -> product_family = "pacemaker"
    if "pacemaker" in filename.lower():
        metadata["product_family"] = ["Pacemaker"]
        metadata["category"] = "Manual"
    elif "defibrillator" in filename.lower():
        metadata["product_family"] = ["Defibrillator"]
        metadata["category"] = "Manual"
    
    # Extract year from filename
    import re
    year_match = re.search(r'20\d{2}', filename)
    if year_match:
        metadata["published_year"] = int(year_match.group())
    
    return metadata

# Test
filename = "pacemaker-5000-2024-manual.pdf"
metadata = extract_custom_metadata(filename)
print(f"Extracted metadata for '{filename}':")
print(f"  {metadata}")

## 6. Index Management

### Creating and Managing Indexes

In [ ]:
import os

# Create index manager
manager = IndexDeploymentManager(
    endpoint=f"https://{os.getenv('AZURE_SEARCH_SERVICE')}.search.windows.net",
    api_key=os.getenv('AZURE_SEARCH_KEY'),
    index_name="my-advanced-index",
    verbose=True
)

print("Index manager created")

### Check if Index Exists

In [ ]:
exists = manager.index_exists()
if exists:
    print("✅ Index exists")
else:
    print("ℹ️  Index does not exist")

### Create Index with Optimized Schema

In [ ]:
# Deploy index (creates if doesn't exist)
success = manager.deploy_index(
    force=False,           # Don't delete if exists
    dry_run=False,         # Actually create
    skip_if_exists=True    # Skip if already exists
)

if success:
    print("✅ Index deployed successfully")
    print("\nIndex includes:")
    print("  - 21 fields (content, metadata, vectors)")
    print("  - 2 scoring profiles (product, content)")
    print("  - Semantic search configuration")
    print("  - Vector search with compression")
    print("  - BM25 similarity tuning")

### Validate Index Configuration

In [ ]:
# Validate index
valid = manager.validate_index()

if valid:
    print("✅ Index validation passed")
else:
    print("⚠️  Index validation found issues")

## 7. Batch Processing Patterns

### Processing Multiple Document Sets

In [ ]:
# Process multiple document types in batches
document_sets = [
    {"glob": "manuals/*.pdf", "category": "Manual"},
    {"glob": "specifications/*.pdf", "category": "Specification"},
    {"glob": "procedures/*.pdf", "category": "Procedure"}
]

total_chunks = 0
total_docs = 0

for doc_set in document_sets:
    print(f"\n📂 Processing {doc_set['category']}...")
    
    config = PipelineConfig.from_env()
    config.input.local.glob = doc_set['glob']
    
    pipeline = Pipeline(config)
    try:
        status = await pipeline.run()
        total_chunks += status.total_chunks_indexed
        total_docs += status.successful_documents
        
        print(f"  ✅ {status.successful_documents} documents")
        print(f"  📦 {status.total_chunks_indexed} chunks")
    finally:
        await pipeline.close()

print(f"\n" + "="*60)
print(f"Total documents: {total_docs}")
print(f"Total chunks: {total_chunks}")
print("="*60)

## 8. Artifact Management

Save processing artifacts for debugging and analysis:

In [ ]:
from ingestor.config import ArtifactsMode

config = PipelineConfig.from_env()

# Enable artifact saving
config.artifacts.mode = ArtifactsMode.LOCAL
config.artifacts.local_path = "./processing_artifacts"

print("Artifacts will be saved to: ./processing_artifacts/")
print("\nArtifact types:")
print("  - {filename}.json: Document Intelligence results")
print("  - {filename}.md: Extracted markdown")
print("  - {filename}_chunks.json: Chunk metadata")

# Process with artifacts
pipeline = Pipeline(config)
try:
    status = await pipeline.run()
    print(f"\n✅ Processing complete with artifacts saved")
finally:
    await pipeline.close()

## 9. Error Handling and Retry Logic

In [ ]:
# Process with custom error handling
config = PipelineConfig.from_env()

# Set retry configuration
config.retry_max_attempts = 3
config.retry_delay_seconds = 2.0

pipeline = Pipeline(config)
try:
    status = await pipeline.run()
    
    # Check for failures
    if status.failed_documents > 0:
        print(f"\n⚠️  {status.failed_documents} documents failed")
        print("\nFailed documents:")
        for result in status.results:
            if not result.success:
                print(f"  - {result.filename}: {result.error_message}")
finally:
    await pipeline.close()

## 10. Performance Monitoring

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Process and collect timing metrics
config = PipelineConfig.from_env()
pipeline = Pipeline(config)

try:
    status = await pipeline.run()
    
    # Analyze performance
    timings = []
    for result in status.results:
        if result.success:
            timings.append({
                "Filename": result.filename,
                "Time (s)": result.processing_time_seconds,
                "Chunks": result.chunks_indexed,
                "Chunks/sec": result.chunks_indexed / result.processing_time_seconds if result.processing_time_seconds > 0 else 0
            })
    
    df = pd.DataFrame(timings)
    
    print("\n📊 Performance Statistics:")
    print(df.describe())
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(df)), df['Time (s)'])
    plt.xlabel('Document')
    plt.ylabel('Processing Time (seconds)')
    plt.title('Document Processing Times')
    plt.show()
    
finally:
    await pipeline.close()

## Summary

You've learned:

✅ Advanced chunking strategies (layout-aware)  
✅ Table and formula processing  
✅ Multi-language support  
✅ Custom metadata extraction  
✅ Index management operations  
✅ Batch processing patterns  
✅ Artifact management  
✅ Error handling and retry logic  
✅ Performance monitoring  

## Next Steps

- **04_real_world_legal.ipynb**: Legal document processing
- **05_real_world_medical.ipynb**: Medical device manual processing
- **07_performance_tuning.ipynb**: Large-scale optimization